#### Se instalan las librerias principales

In [ ]:
!pip install requests
!pip install fastparquet
!pip install pandas

#### Se importan las Librerias

In [1]:
import requests
import pandas as pd
import os

/Users/eduardo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/eduardo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


#### Seccion funciones que extraen de las API

In [2]:
# Esta funcion extrae las coordenadas geograficas de la ciudad ingresada.
# Devuelve un df con los resultados.

def get_city(cityName, country, headers):
    baseUrl = 'https://api.api-ninjas.com/v1/geocoding?city=' + cityName + '&country=' + country
    response = requests.get(baseUrl, headers)
    adress = response.json()
    if len(adress) == 0:
        print('Ciudad no encontrada')
        return pd.DataFrame()
    else:
        return pd.DataFrame.from_dict(adress)

       
def get_weather(cityCoord, fechaStart, fechafin):
    clima = 'https://api.open-meteo.com/v1/forecast?latitude=' + str(cityCoord.iloc[0,1]) + '&longitude=' + str(cityCoord.iloc[0,2]) + '&hourly=temperature_2m,wind_speed_10m&start_date=' + fechaStart + '&end_date=' + fechafin
    
    response = requests.get(clima)
    response_dict = response.json()
    return pd.DataFrame.from_dict(response_dict['hourly'])
    

#### Seccion funciones de verificacion de datos

In [20]:
# Esta funcion intentria verificar si se encuentra mas de una ciudad con el mismo nombre en un mismo pais.
# La dejo para mas adelante que el usuario por ejemplo seleccione el numero de la ciudad deseada.

def check_city(cityCoordDf):
    if len(cityCoordDf) == 0:
        print('Por favor ingrese correctamente el nombre de la ciudad')
    elif len(cityCoordDf) == 1:
        print('Ciudad encontrada correctamente')
        print(cityCoordDf.iloc[0,0])
        return cityCoordDf
    else:
        print(cityCoordDf)
        #return cityCoordDf
        num = int(input("Elija el numero de la ciudad deseada: "))
        #df = pd.DataFrame(cityCoordDf.iloc[num,]).transpose()
        return pd.DataFrame(cityCoordDf.iloc[num,]).transpose()
        

#### Seccion Resguardo de Datos

In [48]:
def saveDataCity(df):
    
    directoryPath = 'weather/' + df.iloc[0,0] + '/'
    dfName = directoryPath + df.iloc[0,0] + '.csv'
    
    directory = os.path.dirname(directoryPath)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    
    df.to_csv(dfName, sep='\t')
       

def saveDataParquet(df, cityCoord, partitionCols=None):
    
    directoryPath = 'weather/' + cityCoord.iloc[0,0] + '/bronce/data'
    
    directory = os.path.dirname(directoryPath)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    
    df.to_parquet(
        directoryPath,
        engine = 'fastparquet')#,
        #partitionCols = partitionCols)
        
    
    

#### Ingreso de datos para extraccion

In [5]:
# Ingreso la ciudad a buscar.
cityName = 'pico truncado'
country = 'argentina'
headers = {'X-Api-Key': 'YO3DOw3APQguBKsqQq0nMA==acw96vJrBQsSnpeG'}

cityCoordDf = get_city(cityName, country, headers)



In [21]:
cityCoord = check_city(cityCoordDf)

print(cityCoord)

            name   latitude  longitude country                state
0  Pico Truncado -46.799193 -67.957653      AR  Santa Cruz Province
1  Pico Truncado -46.799193 -67.957653      AR  Santa Cruz Province
2    Hipotecario -46.805757 -67.956324      AR  Santa Cruz Province
            name   latitude  longitude country                state
0  Pico Truncado -46.799193 -67.957653      AR  Santa Cruz Province


In [23]:
# Genero el directorio para esa ciudad
# Y guardo las coordenadas Geograficas en un archivo csv
saveDataCity(cityCoord)


In [52]:
# Como la ciudad se extrajo con exito
# Ahora consulto el clima

# Antes se consulta las fechas a extraer.
fechaStart = '2024-03-17'
fechafin = '2024-03-25'

weather = get_weather(cityCoord, fechaStart, fechafin)

weather.head

<bound method NDFrame.head of                  time  temperature_2m  wind_speed_10m
0    2024-03-17T00:00            11.3            28.6
1    2024-03-17T01:00            10.4            28.4
2    2024-03-17T02:00             9.6            25.0
3    2024-03-17T03:00             8.1            25.8
4    2024-03-17T04:00             7.3            26.9
..                ...             ...             ...
211  2024-03-25T19:00            16.6            38.9
212  2024-03-25T20:00            16.5            38.8
213  2024-03-25T21:00            16.1            37.8
214  2024-03-25T22:00            15.0            35.2
215  2024-03-25T23:00            13.6            32.4

[216 rows x 3 columns]>

In [45]:
weather.head()

,time,temperature_2m,wind_speed_10m
0,2024-03-17T00:00,11.3,28.6
1,2024-03-17T01:00,10.4,28.4
2,2024-03-17T02:00,9.6,25.0
3,2024-03-17T03:00,8.1,25.8
4,2024-03-17T04:00,7.3,26.9


In [53]:
saveDataParquet(weather,cityCoord)